In [1]:
!wget http://nipg1.inf.elte.hu:8765/MOSI.zip

--2022-01-28 21:42:16--  http://nipg1.inf.elte.hu:8765/MOSI.zip
Resolving nipg1.inf.elte.hu (nipg1.inf.elte.hu)... 157.181.160.161
Connecting to nipg1.inf.elte.hu (nipg1.inf.elte.hu)|157.181.160.161|:8765... connected.
HTTP request sent, awaiting response... 200 OK
Length: 513767740 (490M) [application/zip]
Saving to: ‘MOSI.zip’

MOSI.zip            100%[===================>] 489.97M  23.2MB/s    in 40s     

2022-01-28 21:42:57 (12.1 MB/s) - ‘MOSI.zip’ saved [513767740/513767740]



In [3]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn import preprocessing
import seaborn as sns



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/assignment2'

In [ ]:
x_train = pd.read_csv(drive_path + "x_train.csv").drop(columns=['Unnamed: 0'])
y_train = pd.read_csv(drive_path + "y_train.csv").drop(columns=['Unnamed: 0'])
z_train = pd.read_csv(drive_path + "z_train.csv").drop(columns=['Unnamed: 0'])
x_valid = pd.read_csv(drive_path + "x_valid.csv").drop(columns=['Unnamed: 0'])
y_valid = pd.read_csv(drive_path + "y_valid.csv").drop(columns=['Unnamed: 0'])
z_valid = pd.read_csv(drive_path + "z_valid.csv").drop(columns=['Unnamed: 0'])
x_test = pd.read_csv(drive_path + "x_test.csv").drop(columns=['Unnamed: 0'])
y_test = pd.read_csv(drive_path + "y_test.csv").drop(columns=['Unnamed: 0'])
z_test = pd.read_csv(drive_path + "z_test.csv").drop(columns=['Unnamed: 0'])




In [ ]:
y_test.shape

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,5))
lbls = ['negative', 'positive']

sns.countplot(x=y_train.colimns[0], data=y_train.iloc[:], ax=ax[0]).set_title('training')
ax[0].set_xticklabeles(lbls)

sns.countplot(x=y_valid.colimns[0], data=y_valid.iloc[:], ax=ax[1]).set_title('validation')
ax[1].set_xticklabeles(lbls)

sns.countplot(x=y_test.colimns[0], data=y_test.iloc[:], ax=ax[2]).set_title('testing')
ax[2].set_xticklabeles(lbls)
fig.show()





In [ ]:
x= pd.contact([x_test, x_train, x_valid]).astype(float).values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
x_test = df[0:len(x_test)]
x_train = df[len(x_test):len(x_test)+len(x_train)]
x_valid = df[(len(x_test)+len(x_train)):(len(x_test)+len(x_train)+len(x_valid))]






In [ ]:
from seaborn import categorical
categories = ['0', '1']

categorical_column_sizes = [len(y_test[column])for column in categories]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1//2)) for col_size in categorical_column_sizes)]

y_0 = y_train['0'].values
y_1 = y_train['1'].values
y_train = np.stack([y_0, y_1], 1)
y_train = torch.tensor(y_train, dtype= torch.int64)
zain = torch.tensor(z_train['0'].values).flatten()
x_train = np.stack([x_train[col].values for col in x_train.columns], 1)
x_train = torch.tensor(x_train, dtype=torch.float)


y_0 = y_test['0'].values
y_1 = y_test['1'].values
y_test = np.stack([y_0, y_1], 1)
y_test = torch.tensor(y_test, dtype=torch.int64)
z_test = torch.tensor(z_test['0'].values).flatten()

x_test = np.stack([x_test[col].values for col in x_test.columns], 1)
x_test = torch.tensor(x_test, dtype = torch.float)


In [ ]:
#the model
class Model(nn.Module):
  def __init__(self, cat_emb_size, cat_cols, output_size, layers, dropout):
      super().__init__()
      self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in cat_emb_size])
      self.embeding_dropout = nn.Dropout(dropout)
      self.batch_norm_num = nn.BatchNorm1d(cat_cols)

      model_layers = []
      num_categorical_cols = sum(inf for ni, nf in cat_emb_size)
      input_size = num_categorical_cols + cat_cols

      for i in layers:
        model_layers.append(nn.Linear(input_siza, i))
        model_layers.append(nn.ReLU(inplace = True))
        model_layers.append(nn.BatchNorm1d(i))
        model_layers.append(nn.Dropout(dropout))
        input_size = i
      model_layers.append(nn.Linear(layers[-1], output_size))

      self.layers = nn.Sequential(*model_layers)
  def forward(self, x_categorical, x_numerical):
    embeddings = []
    for i,e in enumerate(self.all_embeddings):
      embeddings.append(e(x_categorical[:, i]))
    x = torch.cat(embeddings, 1)
    x = self.embedding_dropout(x)
    x= numerical =self.batch_norm_num(x_numerical)
    x= torch.cat([x, x_numerical], 1)
    x=self.layers(x)
    return x



In [ ]:
model = Model(categorical_embedding_sizes, x_train.shape[0], 2, [48, 48, 1])
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002)
epochs = 20
aggregated_losses = []
aggregated_valid_losses=[]
aggregated_acc=[]
aggregated_valid_acc=[]
min_loss = np.inf



In [ ]:
for i in range(epochs):
  Y = model(y_train, x_train)
  numerical_pred = np.argmax(Y.detach().numpy(), axis=1)
  correct=len(np.where(numerical_pred==z_train.numpy()[0]))
  aggregated_acc.append(correct/len(z_train))
  single_loss = loss_function(Y, z_train)
  aggregated_losses.append(single_loss.detach().numpy())
  Y = model(y_valid, x_valid)
  numerical_pred = np.argmax(Y.detach().numpy(), axis=1)
  correct = len(np.where(numerical_pred==z_valid.numpy())[0])
  aggregated_valid_acc.append(correct/len(z_valid))
  single_loss = loss_function(Y, z_valid)
  loss = single_loss.detach().numpy()
  aggregated_valid_losses.append(loss)
  optimizer.zero_grad()
  single_loss.backward()
  optimizer.step()
  print(f'epoch: {i+1:3} loss: {single_loss.item():8.6f} acc: {aggregated_acc[-1]:8.6f} valid_acc: {aggregated_valid_acc[-1]:8.6f}')


In [ ]:
plt.plot(range(epochs), aggregated_losses)
plt.plot(range(epochs), aggregated_valid_losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(range(epochs), aggregated_acc)
plt.plot(range(epochs), aggregated_valid_acc)
plt.ylabel('the accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()